In [213]:
# Base de Conocimiento
KB = {
    "Hechos": [],
    "Reglas": [
        {
            "condicion": ["riesgo=bajo", "eficiencia_energetica=True"], 
            "decision": "Aprobado",
            "justificacion": "Bajo riesgo con alta eficiencia energética",
        },
        {
            "condicion": ["riesgo=alto", "impacto_ambiental=negativo"],
            "decision": "Rechazado",
            "justificacion": "Alto riesgo con impacto ambiental negativo",
        },
        {
            "condicion": ["endeudamiento=medio", "impacto_ambiental=moderado"],
            "decision": "AprobarCondicionadoGarantia",
            "justificacion": "Endeudamiento medio con impacto ambiental moderado",
        }
    ],
    "ReglasCombinadas": [
        {
            "condicion": ["riesgo=medio", "impacto_ambiental=positivo", "cofinanciamiento=True"], 
            "decision": "AprobadoCondicionadoMitigaciónSeguimiento",
            "justificacion": "Riesgo medio mitigado por cofinanciamiento y impacto ambiental positivo",
            "FactoresAdicionales": ["ApoyoComunitario"],
        },
        {
            "condicion": ["riesgo=alto", "eficiencia_energetica=False", "impacto_ambiental=negativo"],
            "decision": "RechazadoDefinitivo",
            "justificacion": "Múltiples factores de riesgo: alto riesgo, baja eficiencia energética e impacto negativo",
            "FactoresAdicionales": ["RechazoComunitario"],
        },
    ],
    "RegistroAuditoria": []
}

In [214]:
# Funciones Auxiliares
def verificar_condicion(hechos, condicion):
    """Verifica si todas las condiciones se cumplen en los hechos"""
    for condic in condicion:
        if condic not in hechos:
            return False
    return True

def verificar_factores(factores_adicionales, factores_requeridos):
    """Verifica si los factores adicionales contienen los requeridos"""
    for factor in factores_requeridos:
        if not any(factor.lower() in h.lower() for h in factores_adicionales):
            return False
    return True

def agregar_hecho(solicitante, proyecto, condicion, valor):
    """Agrega un hecho a la base de conocimiento"""
    KB["Hechos"].append({
        "solicitante": solicitante,
        "proyecto": proyecto,
        "condicion": condicion,
        "valor": valor
    })

def hechos_de(solicitante, proyecto):
    """Obtiene todos los hechos de un solicitante y proyecto específico"""
    return [h for h in KB["Hechos"] if h["solicitante"] == solicitante and h["proyecto"] == proyecto]

In [215]:
# Recopilación de Datos
def recopilar_datos(info_cliente):
    """Recopila datos del cliente y los convierte en hechos"""
    hechos = {info: True for info in info_cliente}
    # print("Datos recopilados:", info_cliente)
    return hechos

In [216]:
def aplicar_reglas_especificas(KB, hechos):
    """Aplica reglas específicas y retorna resultados con justificación"""
    resultado = []
    for i, regla in enumerate(KB["Reglas"], 1):
        if verificar_condicion(hechos, regla["condicion"]):
            resultado.append({
                "decision": regla["decision"],
                "condicion": regla["condicion"],
                "justificacion": regla.get("justificacion", ""),
                "acciones": regla.get("acciones", []),
                "regla_num": i,
                "tipo": "especifica"
            })
            # print(f"Regla específica #{i} activada: {regla['decision']}")
    return resultado

def aplicar_reglas_combinadas(KB, hechos, factores_adicionales):
    """Aplica reglas combinadas y retorna resultados con justificación"""
    resultados_combinados = []
    for i, regla in enumerate(KB["ReglasCombinadas"], 1):
        if (verificar_condicion(hechos, regla["condicion"]) and
                verificar_factores(factores_adicionales, regla["FactoresAdicionales"])):
            resultados_combinados.append({
                "decision": regla["decision"],
                "condicion": regla["condicion"],
                "justificacion": regla.get("justificacion", ""),
                "acciones": regla.get("acciones", []),
                "regla_num": i,
                "tipo": "combinada"
            })
            print(f"Regla combinada #{i} activada: {regla['decision']}")
    return resultados_combinados

In [217]:
def priorizar_decisiones(decisiones, solicitante, proyecto):
    """Prioriza las decisiones y retorna ordenadas por relevancia"""
    return sorted(decisiones, 
                  key=lambda x: calcular_puntaje_decision(x, solicitante, proyecto), 
                  reverse=True)

def calcular_puntaje_decision(decision, solicitante, proyecto):
    """
    Calcula un puntaje para priorizar decisiones de crédito
    """
    puntaje = 0
    decision_text = decision.get('decision', str(decision))
    # Nueva lógica: bases y bonificaciones ajustadas
    if decision_text == "Aprobado":
        puntaje += 60  # Base alta para aprobado
        if tiene_historial_positivo(solicitante):
            puntaje += 7
        if es_proyecto_prioritario(proyecto):
            puntaje += 5
        if tiene_impacto_ambiental_positivo(solicitante, proyecto):
            puntaje += 3
    elif "Condicionado" in decision_text:
        puntaje += 30  # Base baja para condicionado
        if tiene_historial_positivo(solicitante):
            puntaje += 7
        if es_proyecto_prioritario(proyecto):
            puntaje += 5
        if tiene_impacto_ambiental_positivo(solicitante, proyecto):
            puntaje += 2
    else:
        # Otros casos (ejemplo: Rechazado)
        puntaje = 0
    return puntaje

def seleccionar_acciones_por_puntaje(puntaje):
    """Selecciona acciones dinámicas basadas en el puntaje obtenido"""
    
    # Acciones según rangos de puntaje
    if puntaje >= 60:
        # MÁXIMA PRIORIDAD - Acciones premium
        acciones_dinamicas = [
            "✅ Aprobación expedita (48 horas)",
            "✅ Desembolso acelerado",
            "✅ Seguimiento técnico trimestral",
            "✅ Acompañamiento especializado en implementación",
            "✅ Acceso a líneas de financiamiento complementarias",
            "✅ Exención de garantías adicionales",
            "✅ Tasa de interés preferencial aplicable"
        ]
    elif puntaje >= 45:
        # PRIORIDAD ALTA - Acciones favorables
        acciones_dinamicas = [
            "✅ Aprobación estándar (5 días hábiles)",
            "✅ Desembolso con trámites normales",
            "✅ Seguimiento técnico semestral",
            "✅ Auditoría de cofinanciamiento",
            "✅ Reporte trimesal de avances",
            "✅ Garantías reducidas (50% de lo solicitado)"
        ]
    elif puntaje >= 30:
        # PRIORIDAD MEDIA - Acciones condicionales
        acciones_dinamicas = [
            "⚠️ Aprobación condicionada (10 días hábiles)",
            "⚠️ Desembolso por etapas según hitos",
            "⚠️ Seguimiento técnico mensual",
            "⚠️ Solicitar garantías adicionales",
            "⚠️ Revisión manual por comité de crédito",
            "⚠️ Auditoría inicial de proyecto"
        ]
    elif puntaje >= 15:
        # PRIORIDAD BAJA - Acciones restrictivas
        acciones_dinamicas = [
            "❌ Aprobación requiere revisión profunda (15 días)",
            "❌ Desembolso en múltiples tramos con hitos",
            "❌ Seguimiento técnico semanal",
            "❌ Garantías totales al 100%",
            "❌ Revisión manual obligatoria",
            "❌ Evaluación ambiental detallada",
            "❌ Plan de mitigación de riesgos requerido"
        ]
    else:
        # RECHAZO - Acciones de cierre
        acciones_dinamicas = [
            "❌ RECHAZO DE SOLICITUD",
            "❌ Cierre de expediente",
            "❌ Notificación formal al solicitante",
            "❌ Registro en base de datos de rechazados",
            "❌ Retroalimentación regulatoria"
        ]
    
    return acciones_dinamicas

def tiene_historial_positivo(solicitante):
    """
    Verifica si el solicitante tiene un historial crediticio positivo
    """
    # Buscar en los hechos del solicitante por indicadores positivos
    hechos_solicitante = [h for h in KB["Hechos"] if h["solicitante"] == solicitante]
    
    for hecho in hechos_solicitante:
        if (hecho.get("condicion") == "riesgo" and hecho.get("valor") in ["bajo", "medio"]) or \
           (hecho.get("condicion") == "historial_crediticio" and hecho.get("valor") == "bueno"):
            return True
    return False

def es_proyecto_prioritario(proyecto):
    """
    Determina si el proyecto es de alta prioridad ambiental
    """
    proyectos_prioritarios = ["Planta Solar", "Edificio Verde", "Parque Eólico", "Sistema Geotérmico"]
    return any(prioritario.lower() in proyecto.lower() for prioritario in proyectos_prioritarios)

def tiene_impacto_ambiental_positivo(solicitante, proyecto):
    """
    Verifica si el proyecto tiene impacto ambiental positivo
    """
    hechos_proyecto = hechos_de(solicitante, proyecto)
    
    for hecho in hechos_proyecto:
        if (hecho.get("condicion") == "impacto_ambiental" and hecho.get("valor") == "positivo") or \
           (hecho.get("condicion") == "eficiencia_energetica" and hecho.get("valor") is True):
            return True
    return False

In [218]:
def generar_reporte_completo(solicitante, proyecto, decisiones_priorizadas, hechos_str):
    """Genera un reporte completo con alertas y recomendaciones"""
    
    if not decisiones_priorizadas:
        return "No se encontraron decisiones aplicables."
    
    # Obtener la decisión de mayor prioridad
    decision_final = decisiones_priorizadas[0]
    decision_text = decision_final.get('decision', str(decision_final))
    justificacion = decision_final.get('justificacion', 'Sin justificación disponible')
    puntaje_final = calcular_puntaje_decision(decision_final, solicitante, proyecto)
    
    # Seleccionar acciones dinámicas basadas en el puntaje
    acciones_finales = seleccionar_acciones_por_puntaje(puntaje_final)
    
    # Construir el reporte
    reporte = []
    reporte.append("\n" + "="*70)
    reporte.append(f"REPORTE DE EVALUACIÓN DE CRÉDITO VERDE")
    reporte.append("="*70)
    reporte.append(f"\nSOLICITANTE: {solicitante}")
    reporte.append(f"PROYECTO: {proyecto}")
    reporte.append(f"FECHA: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Sección 1: HECHOS EVALUADOS
    reporte.append(f"\n{'-'*70}")
    reporte.append("1. HECHOS EVALUADOS")
    reporte.append(f"{'-'*70}")
    for hecho in hechos_str:
        reporte.append(f"   • {hecho}")
    
    # Sección 2: DECISIÓN AUTOMATIZADA
    reporte.append(f"\n{'-'*70}")
    reporte.append("2. DECISIÓN AUTOMATIZADA")
    reporte.append(f"{'-'*70}")
    
    # Asignar símbolo según tipo de decisión
    if "Rechazado" in decision_text:
        simbolo = "❌ RECHAZADO"
        estatus = "NEGATIVA"
    elif "Aprobado" in decision_text and "Condicionado" not in decision_text:
        simbolo = "✅ APROBADO"
        estatus = "POSITIVA"
    else:
        simbolo = "⚠️ CONDICIONADO"
        estatus = "CONDICIONAL"
    
    reporte.append(f"\n   {simbolo}")
    reporte.append(f"   Estatus: {estatus}")
    reporte.append(f"   Decisión: {decision_text}")
    
    # Sección 3: JUSTIFICACIÓN
    reporte.append(f"\n{'-'*70}")
    reporte.append("3. JUSTIFICACIÓN BASADA EN REGLAS")
    reporte.append(f"{'-'*70}")
    reporte.append(f"   {justificacion}")

    
    # Sección 5: ACCIONES REQUERIDAS (DINÁMICAS SEGÚN PUNTAJE)
    reporte.append(f"\n{'-'*70}")
    reporte.append("4. ACCIONES REQUERIDAS (SEGÚN PUNTAJE)")
    reporte.append(f"{'-'*70}")
    
    # Mostrar rango de puntaje
    if puntaje_final >= 60:
        rango = "MÁXIMA PRIORIDAD - Acciones Premium"
    elif puntaje_final >= 45:
        rango = "PRIORIDAD ALTA - Acciones Favorables"
    elif puntaje_final >= 30:
        rango = "PRIORIDAD MEDIA - Acciones Condicionales"
    elif puntaje_final >= 15:
        rango = "PRIORIDAD BAJA - Acciones Restrictivas"
    else:
        rango = "RECHAZO - Acciones de Cierre"
    
    reporte.append(f"\n   🎯 Rango de Prioridad: {rango}\n")
    
    if acciones_finales:
        for i, accion in enumerate(acciones_finales, 1):
            reporte.append(f"   {i}. {accion}")
    else:
        reporte.append("   • No se requieren acciones adicionales")
    
    # Sección 6: ALERTAS Y RECOMENDACIONES
    reporte.append(f"\n{'-'*70}")
    reporte.append("5. ALERTAS Y RECOMENDACIONES")
    reporte.append(f"{'-'*70}")
    
    alertas = generar_alertas(decision_text, hechos_str, solicitante, proyecto)
    for alerta in alertas:
        reporte.append(f"   ⚡ {alerta}")
    
    # Sección 7: REGISTRO DE TRAZABILIDAD
    reporte.append(f"\n{'-'*70}")
    reporte.append("6. REGISTRO DE TRAZABILIDAD (AUDITORÍA)")
    reporte.append(f"{'-'*70}")
    (solicitante, proyecto, decision_text, hechos_str, acciones_finales)
    reporte.append(f"   ✓ Evaluación registrada en sistema de auditoría")
    reporte.append(f"   ✓ ID de seguimiento: AUDIT-{solicitante[:3]}-{proyecto[:3]}-{__import__('datetime').datetime.now().strftime('%Y%m%d%H%M%S')}")
    reporte.append(f"   ✓ Disponible para retroalimentación regulatoria")
    
    reporte.append(f"\n{'='*70}\n")
    
    return "\n".join(reporte)

def generar_alertas(decision_text, hechos_str, solicitante, proyecto):
    """Genera alertas contextuales basadas en la decisión y hechos"""
    alertas = []
    
    # Análisis de riesgo
    if "riesgo=alto" in hechos_str:
        alertas.append("RIESGO ELEVADO: Se requiere revisión manual obligatoria antes de procesar")
    
    if "riesgo=medio" in hechos_str:
        alertas.append("RIESGO MEDIO: Se recomienda seguimiento técnico periódico del proyecto")
    
    # Análisis ambiental
    if "impacto_ambiental=negativo" in hechos_str:
        alertas.append("IMPACTO AMBIENTAL NEGATIVO: Proyecto no alineado con objetivos de sostenibilidad")
    
    if "impacto_ambiental=positivo" in hechos_str:
        alertas.append("IMPACTO AMBIENTAL POSITIVO: Proyecto contribuye a objetivos de sostenibilidad")
    
    # Análisis de eficiencia
    if "eficiencia_energetica=False" in hechos_str:
        alertas.append("EFICIENCIA ENERGÉTICA BAJA: Requiere mejoras técnicas o revisión de viabilidad")
    
    if "eficiencia_energetica=True" in hechos_str:
        alertas.append("EFICIENCIA ENERGÉTICA ALTA: Proyecto cumple estándares internacionales")
    
    # Análisis de decisión
    if "Rechazado" in decision_text:
        alertas.append("DECISIÓN NEGATIVA: Proyecto no cumple criterios mínimos de aprobación")
        alertas.append("ACCIÓN: Notificar al solicitante con feedback detallado para mejora")
    
    if "Condicionado" in decision_text:
        alertas.append("DECISIÓN CONDICIONAL: Aprobación sujeta a cumplimiento de condiciones")
        alertas.append("ACCIÓN: Establecer hitos de monitoreo y plazos de cumplimiento")
    
    if decision_text == "Aprobado":
        alertas.append("DECISIÓN POSITIVA: Proyecto cumple todos los criterios de aprobación")
        alertas.append("ACCIÓN: Proceder con trámites administrativos y desembolso")
    
    # Análisis de cofinanciamiento
    if "cofinanciamiento=True" in hechos_str:
        alertas.append("COFINANCIAMIENTO PRESENTE: Riesgo compartido con otras instituciones")
    
    return alertas if alertas else ["Sin alertas relevantes"]


In [219]:
# ==================== FLUJO PRINCIPAL ====================

def evaluar_credito_completo(info_cliente, solicitante, proyecto):
    """Función principal de evaluación con reporte completo"""
    
    hechos = recopilar_datos(info_cliente)
    
    # Aplicar reglas específicas y combinadas
    res_especificas = aplicar_reglas_especificas(KB, hechos)
    res_combinadas = aplicar_reglas_combinadas(KB, hechos, info_cliente)
    
    # Combinar todas las decisiones
    todas_decisiones = res_especificas + res_combinadas
    
    # Priorizar decisiones
    if todas_decisiones:
        decisiones_priorizadas = priorizar_decisiones(todas_decisiones, solicitante, proyecto)
        
        # Generar y mostrar reporte completo
        reporte = generar_reporte_completo(solicitante, proyecto, decisiones_priorizadas, info_cliente)
        print(reporte)
        
        return decisiones_priorizadas
    else:
        print(f"\nNo se encontraron decisiones aplicables para {solicitante} - {proyecto}")
        return []

In [220]:
def main():
    KB["Hechos"] = []
    
    # Datos de prueba
    datos = [
        {
            "solicitante": "Juan García",
            "proyecto": "Edificio Verde Downtown",
            "hechos": [
                ("riesgo", "bajo"),
                ("eficiencia_energetica", True)
            ]
        },
        {
            "solicitante": "Ana López",
            "proyecto": "Parque Eólico Costero",
            "hechos": [
                ("riesgo", "alto"),
                ("impacto_ambiental", "negativo")
            ]
        },
        {
            "solicitante": "Luis Martínez",
            "proyecto": "Sistema Geotérmico Industrial",
            "hechos": [
                ("riesgo", "medio"),
                ("impacto_ambiental", "positivo"),
                ("cofinanciamiento", True),
                ("ApoyoComunitario", True)
            ]
        }
    ]
    
    # Procesar cada solicitud
    for d in datos:
        # Agregar hechos a la base de conocimiento
        for cond, val in d["hechos"]:
            agregar_hecho(d["solicitante"], d["proyecto"], cond, val)
        
        # Obtener hechos formateados
        hechos = hechos_de(d["solicitante"], d["proyecto"])
        hechos_str = [f"{h['condicion']}={h['valor']}" for h in hechos]
        
        # Evaluar crédito con reporte completo
        evaluar_credito_completo(hechos_str, d["solicitante"], d["proyecto"])

if __name__ == "__main__":
    main()


REPORTE DE EVALUACIÓN DE CRÉDITO VERDE

SOLICITANTE: Juan García
PROYECTO: Edificio Verde Downtown
FECHA: 2025-10-14 23:25:05

----------------------------------------------------------------------
1. HECHOS EVALUADOS
----------------------------------------------------------------------
   • riesgo=bajo
   • eficiencia_energetica=True

----------------------------------------------------------------------
2. DECISIÓN AUTOMATIZADA
----------------------------------------------------------------------

   ✅ APROBADO
   Estatus: POSITIVA
   Decisión: Aprobado

----------------------------------------------------------------------
3. JUSTIFICACIÓN BASADA EN REGLAS
----------------------------------------------------------------------
   Bajo riesgo con alta eficiencia energética

----------------------------------------------------------------------
4. ACCIONES REQUERIDAS (SEGÚN PUNTAJE)
----------------------------------------------------------------------

   🎯 Rango de Prioridad: MÁXI